//+------------------------------------------------------------------+
//|                                                      H1_RSI_EA.mq5 |
//|                        Generated by GPT-5-mini                     |
//+------------------------------------------------------------------+
#property strict

input double RiskPercent = 1.0;        // Risk per trade in %
input int RSI_Period = 14;             // RSI period
input double RSI_BuyTP = 70;           // RSI overbought for Buy TP
input double RSI_SellTP = 30;          // RSI oversold for Sell TP

// Global variables
datetime lastTradeTime = 0;

//+------------------------------------------------------------------+
//| Expert initialization function                                   |
//+------------------------------------------------------------------+
int OnInit()
{
   return(INIT_SUCCEEDED);
}

//+------------------------------------------------------------------+
//| Expert tick function                                             |
//+------------------------------------------------------------------+
void OnTick()
{
   static MqlRates rates[];
